In [9]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.compat.v1.Session(config=config)

In [2]:
def create_gen_input(self, batch_size=32, noise_size=62, n_class=10, seed=None):
  # create noise input
  noise = tf.random.normal([batch_size, noise_size], seed=seed)
  # Create categorical latent code
  label = tf.random.uniform([batch_size], minval=0, maxval=10, dtype=tf.int32, seed=seed)
  label = tf.one_hot(label, depth=n_class)
  # Create one continuous latent code
  c_1 = tf.random.uniform([batch_size, 1], minval=-1, maxval=1, seed=seed)
  return label, c_1, noise

In [3]:
def create_generator_continuous(n_filters=128, input_size=73):
    # Build functional API model
    # input
    input = keras.layers.Input(shape=(input_size, ))

    # Fully-connected layer.
    dense_1 = keras.layers.Dense(units=1024, use_bias=False) (input)
    bn_1 = keras.layers.BatchNormalization()(dense_1)
    act_1 = keras.layers.ReLU()(bn_1)
    # Fully-connected layer. The output should be able to reshape into 7x7
    dense_2 = keras.layers.Dense(units=7*7*128, use_bias=False) (act_1)
    bn_2 = keras.layers.BatchNormalization()(dense_2)
    act_2 = keras.layers.ReLU()(bn_2)
    # Reshape
    reshape = keras.layers.Reshape(target_shape=(7, 7, 128))(act_2)

    nf = n_filters
    # First transposed convolutional layer

    tc_1 = keras.layers.Conv2DTranspose(nf, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)(reshape)
    bn_1 = keras.layers.BatchNormalization()(tc_1)
    act_1 = keras.layers.ReLU()(bn_1)

    # Number of filters halved after each transposed convolutional layer
    nf = nf//2
    # Second transposed convolutional layer
    # strides=(2, 2): shape is doubled after the transposed convolution
    tc_2 = keras.layers.Conv2DTranspose(nf, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)(act_1)
    bn_2 = keras.layers.BatchNormalization()(tc_2)
    act_2 = keras.layers.ReLU()(bn_2)

    # Final transposed convolutional layer: output shape: 28x28x1, tanh activation
    output = keras.layers.Conv2DTranspose(1, kernel_size=(4, 4), strides=(1, 1), 
                                         padding="same", activation="tanh")(act_2)
    model = keras.models.Model(inputs=input, outputs=output)
    return model

In [4]:
def create_discriminator_continuous(n_filters=64, n_class=10, input_shape=(28, 28, 1)):
    # Build functional API model
    # Image Input
    image_input = keras.layers.Input(shape=input_shape)

    nf = n_filters
    c_1 = keras.layers.Conv2D(nf, kernel_size=(4, 4), strides=(2, 2), padding="same", use_bias=True)(image_input)
    bn_1 = keras.layers.BatchNormalization()(c_1)
    act_1 = keras.layers.LeakyReLU(alpha=0.1)(bn_1)

    # Number of filters doubled after each convolutional layer
    nf = nf*2
    # Second convolutional layer
    # Output shape: 7x7
    c_2 = keras.layers.Conv2D(nf, kernel_size=(4, 4), strides=(2, 2), padding="same", use_bias=False)(act_1)
    bn_2 = keras.layers.BatchNormalization()(c_2)
    act_2 = keras.layers.LeakyReLU(alpha=0.1)(bn_2)

    # Flatten the convolutional layers
    flatten = keras.layers.Flatten()(act_2)

    # FC layer
    dense = keras.layers.Dense(1024, use_bias=False)(flatten)
    bn = keras.layers.BatchNormalization()(dense)
    act = keras.layers.LeakyReLU(alpha=0.1)(bn)
    # Discriminator output. Sigmoid activation function to classify "True" or "False"
    d_output = keras.layers.Dense(1, activation='sigmoid')(act)

    # Auxiliary output. 
    q_dense = keras.layers.Dense(128, use_bias=False)(act)
    q_bn = keras.layers.BatchNormalization()(q_dense)
    q_act = keras.layers.LeakyReLU(alpha=0.1)(q_bn)

    # Classification (discrete output)
    clf_out = keras.layers.Dense(n_class, activation="softmax")(q_act)

    # Gaussian distribution mean (continuous output)
    mu = keras.layers.Dense(1)(q_act)

    # Gaussian distribution standard deviation (exponential activation to ensure the value is positive)
    sigma = keras.layers.Dense(1, activation=lambda x: tf.math.exp(x))(q_act)

    # Discriminator model (not compiled)
    d_model = keras.models.Model(inputs=image_input, outputs=d_output)

    # Auxiliary model (not compiled)
    q_model = keras.models.Model(inputs=image_input, outputs=[clf_out, mu, sigma])
    return d_model, q_model

In [5]:
class InfoGAN_Continuous(keras.Model):
    def __init__(self, d_model, g_model, q_model,noise_size, num_classes):
        super(InfoGAN_Continuous, self).__init__()
        self.d_model = d_model
        self.g_model = g_model
        self.q_model = q_model
        self.noise_size = noise_size
        self.num_classes = num_classes

    def compile(self, d_optimizer, g_optimizer, q_optimizer):
        super(InfoGAN_Continuous, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.q_optimizer = q_optimizer

    def create_gen_input(self, batch_size, noise_size, n_class, seed=None):
        # create noise input
        noise = tf.random.normal([batch_size, noise_size], seed=seed)
        # Create categorical latent code
        label = tf.random.uniform([batch_size], minval=0, maxval=10, dtype=tf.int32, seed=seed)
        label = tf.one_hot(label, depth=n_class)
        # Create one continuous latent code
        c_1 = tf.random.uniform([batch_size, 1], minval=-1, maxval=1, seed=seed)
        return label, c_1, noise

    def concat_inputs(self, input):
        concat_input = keras.layers.Concatenate()(input)
        return concat_input

    def train_step(self, real_image_batch):
        # Define loss functions
        binary_loss = keras.losses.BinaryCrossentropy()
        categorical_loss = keras.losses.CategoricalCrossentropy()
        # Half-batch for training discriminator and batch for training generator and auxiliary model
        batch = tf.shape(real_image_batch)[0]
        # Create generator input 
        g_label, c_1, g_noise = self.create_gen_input(batch, self.noise_size, self.num_classes, seed=None)
        g_input = self.concat_inputs([g_label, c_1, g_noise])
        with tf.GradientTape() as d_tape: 
            self.d_model.trainable = True
            d_tape.watch(self.d_model.trainable_variables)
            # Train discriminator using half batch real images
            y_disc_real = tf.ones((batch, 1))
            d_real_output = self.d_model(real_image_batch, training=True)
            d_loss_real = binary_loss(y_disc_real, d_real_output)
            # Train discriminator using half batch fake images     
            y_disc_fake = tf.zeros((batch, 1))
            # Create fake image batch
            fake_image_batch = self.g_model(g_input, training=True)
            d_fake_output = self.d_model(fake_image_batch, training=True)
            d_loss_fake = binary_loss(y_disc_fake, d_fake_output)
            d_loss = d_loss_real + d_loss_fake
        # Calculate gradients
        d_gradients = d_tape.gradient(d_loss, self.d_model.trainable_variables)
        # Optimize
        self.d_optimizer.apply_gradients(zip(d_gradients, self.d_model.trainable_variables))
        with tf.GradientTape() as g_tape, tf.GradientTape() as q_tape:
            # Create generator input 
            g_label, c_1, g_noise = self.create_gen_input(batch*2, self.noise_size, self.num_classes, seed=None)
            g_input = self.concat_inputs([g_label, c_1, g_noise])
            g_tape.watch(self.g_model.trainable_variables)
            q_tape.watch(self.q_model.trainable_variables)
            # Create fake image batch
            fake_image_batch = self.g_model(g_input, training=True)
            d_fake_output = self.d_model(fake_image_batch, training=True)
            # Generator Image loss
            y_gen_fake = tf.ones((batch*2, 1))
            g_img_loss = binary_loss(y_gen_fake, d_fake_output)
            # Auxiliary loss
            cat_output, mu, sigma = self.q_model(fake_image_batch, training=True)
            # Categorical loss
            cat_loss = categorical_loss(g_label, cat_output)
            # Use Gaussian distributions to represent the output
            dist = tfp.distributions.Normal(loc=mu, scale=sigma)
            # Losses (negative log probability density function as we want to maximize the probability density function)
            c_1_loss = tf.reduce_mean(-dist.log_prob(c_1))
            # Generator total loss
            g_loss = g_img_loss + (cat_loss + 0.1*c_1_loss)
            # Auxiliary function loss
            q_loss = (cat_loss + 0.1*c_1_loss)
        # Calculate gradients
        # We do not want to modify the neurons in the discriminator when training the generator and the auxiliary model
        self.d_model.trainable=False
        g_gradients = g_tape.gradient(g_loss, self.g_model.trainable_variables)
        q_gradients = q_tape.gradient(q_loss, self.q_model.trainable_variables)
        # Optimize
        self.g_optimizer.apply_gradients(zip(g_gradients, self.g_model.trainable_variables))
        self.q_optimizer.apply_gradients(zip(q_gradients, self.q_model.trainable_variables))

        return {"d_loss_real": d_loss_real, "d_loss_fake": d_loss_fake, "g_img_loss": g_img_loss ,
                "cat_loss": cat_loss, "c_1_loss": c_1_loss}

In [10]:
def load_real_image(batch_size=32):
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
    # Add the color channel - change to 4D tensor, and convert the data type to 'float32'
    train_images = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
    # Set the pixel values from -1 to 1
    train_images = (train_images/255.0) * 2 - 1
    # Shuffle and separate in batch
    buffer_size = train_images.shape[0]
    train_images_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(buffer_size).batch(batch_size)
    return train_images_dataset

g_model_continuous = create_generator_continuous()
d_model_continuous, q_model_continuous = create_discriminator_continuous()
infogan = InfoGAN_Continuous(d_model_continuous, g_model_continuous, q_model_continuous, noise_size=62, num_classes=10)
infogan.compile(d_optimizer=keras.optimizers.Adam(learning_rate=2e-4),
                g_optimizer=keras.optimizers.Adam(learning_rate=5e-4),
                q_optimizer=keras.optimizers.Adam(learning_rate=2e-4))
real_images = load_real_image(batch_size=32)
infogan.fit(real_images, epochs=1)

1875/1875 [==============================] - 1330s 707ms/step - d_loss_real: 0.4781 - d_loss_fake: 0.4628 - g_img_loss: 1.2399 - cat_loss: 0.2621 - c_1_loss: 0.8898


In [13]:
def generate_fake_samples(generator, n_samples):
        # generate points in latent space and control codes
        z_input, _ = generate_noise(n_samples)
        # predict outputs
        images = generator.predict(z_input)
        # create class labels
        y = zeros((n_samples, 1))
        return images, y
    
def generate_noise(n_samples):
        # generate points in the latent space
        z_latent=randn(n_samples,10)
        # generate categorical one-hot codes
        cat_codes=np.eye(self.n_cat)[np.random.choice(self.n_cat, n_samples)]
        # concatenate latent points and control codes
        z_input = hstack((z_latent, cat_codes))
        return [z_input, cat_codes]

In [12]:
image, y = generate_fake_samples(infogan.g_model, 10)

TypeError: generate_fake_samples() missing 1 required positional argument: 'n_samples'